# 🐄 Cattle & Breed Classification Model Training

This notebook provides a comprehensive guide for training deep learning models for cattle species and breed classification using the Indian Bovine Breeds dataset from Kaggle.

## 📋 Training Overview

- **Dataset**: [Indian Bovine Breeds Dataset](https://www.kaggle.com/datasets/lukex9442/indian-bovine-breeds)
- **Models**: Two-stage classification system
  1. **Cattle Classifier**: Cow vs Buffalo vs None (3 classes)
  2. **Breed Classifier**: 41 different breeds
- **Architecture**: ResNet-18 with transfer learning
- **Framework**: PyTorch

## 🎯 Training Objectives

1. Achieve 95%+ accuracy for cattle classification
2. Achieve 88%+ accuracy for breed classification  
3. Optimize models for real-time inference
4. Save production-ready model weights

## 1. Environment Setup and Dependencies

First, let's install and import all required libraries for training our cattle classification models.

In [ ]:
# Install required packages (run this if packages are not installed)
# !pip install torch torchvision Pillow matplotlib seaborn pandas numpy scikit-learn tqdm kaggle

# Import essential libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import transforms, models
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import os
import shutil
from pathlib import Path
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

## 2. Dataset Loading and Preprocessing

Let's download the Indian Bovine Breeds dataset from Kaggle and set up our data preprocessing pipeline.

In [ ]:
# Download dataset from Kaggle
# First, you need to set up Kaggle API credentials
# 1. Go to kaggle.com -> Account -> API -> Create New API Token
# 2. Place kaggle.json in ~/.kaggle/ directory
# 3. Run: chmod 600 ~/.kaggle/kaggle.json

# Uncomment and run the following lines to download the dataset:
# !kaggle datasets download -d lukex9442/indian-bovine-breeds
# !unzip -q indian-bovine-breeds.zip -d ./data/

# Define data directories
data_dir = Path('./data')
dataset_dir = data_dir / 'indian-bovine-breeds'

# Create directories if they don't exist
data_dir.mkdir(exist_ok=True)
models_dir = Path('./models')
models_dir.mkdir(exist_ok=True)

print(f"Data directory: {data_dir}")
print(f"Dataset directory: {dataset_dir}")
print(f"Models directory: {models_dir}")

# Check if dataset exists
if dataset_dir.exists():
    print(f"✅ Dataset found at {dataset_dir}")
    # List dataset contents
    for item in sorted(dataset_dir.iterdir()):
        if item.is_dir():
            print(f"📁 {item.name}: {len(list(item.glob('*')))} items")
else:
    print("❌ Dataset not found. Please download the dataset first.")

In [ ]:
# Define breed classes based on README
INDIAN_BREEDS = [
    'Alambadi', 'Amritmahal', 'Banni', 'Bargur', 'Bhadawari', 'Dangi', 'Deoni', 
    'Gir', 'Hallikar', 'Hariana', 'Jaffrabadi', 'Kangayam', 'Kankrej', 'Kasargod', 
    'Kenkatha', 'Kherigarh', 'Khillari', 'Krishna Valley', 'Malnad Gidda', 'Mehsana', 
    'Murrah', 'Nagori', 'Nagpuri', 'Nili Ravi', 'Nimari', 'Ongole', 'Pulikulam', 
    'Rathi', 'Red Sindhi', 'Sahiwal', 'Surti', 'Tharparkar', 'Toda', 'Umblachery', 'Vechur'
]

INTERNATIONAL_BREEDS = [
    'Ayrshire', 'Brown Swiss', 'Guernsey', 'Holstein Friesian', 'Jersey', 'Red Dane'
]

ALL_BREEDS = INDIAN_BREEDS + INTERNATIONAL_BREEDS
CATTLE_CLASSES = ['Cow', 'Buffalo', 'None']

print(f"Total breeds: {len(ALL_BREEDS)}")
print(f"Indian breeds: {len(INDIAN_BREEDS)}")
print(f"International breeds: {len(INTERNATIONAL_BREEDS)}")
print(f"Cattle classes: {CATTLE_CLASSES}")

# Create class to index mappings
breed_to_idx = {breed: idx for idx, breed in enumerate(ALL_BREEDS)}
idx_to_breed = {idx: breed for breed, idx in breed_to_idx.items()}

cattle_to_idx = {cattle: idx for idx, cattle in enumerate(CATTLE_CLASSES)}
idx_to_cattle = {idx: cattle for cattle, idx in cattle_to_idx.items()}

print(f"\nBreed classes: {len(breed_to_idx)}")
print(f"Cattle classes: {len(cattle_to_idx)}")

In [ ]:
# Custom Dataset Class
class CattleBreedDataset(Dataset):
    def __init__(self, root_dir, transform=None, task='breed'):
        """
        Args:
            root_dir (string): Directory with all the images organized by class
            transform (callable, optional): Optional transform to be applied on a sample
            task (string): 'breed' for breed classification, 'cattle' for cattle classification
        """
        self.root_dir = Path(root_dir)
        self.transform = transform
        self.task = task
        self.samples = []
        self.classes = []
        
        if task == 'breed':
            self.class_to_idx = breed_to_idx
            self.classes = ALL_BREEDS
        else:  # cattle classification
            self.class_to_idx = cattle_to_idx
            self.classes = CATTLE_CLASSES
        
        # Load all image paths and labels
        self._load_samples()
        
    def _load_samples(self):
        """Load all image paths and corresponding labels"""
        for class_name in self.classes:
            class_dir = self.root_dir / class_name
            if class_dir.exists():
                for img_path in class_dir.glob('*.jpg'):
                    if img_path.is_file():
                        self.samples.append((str(img_path), self.class_to_idx[class_name]))
                        
        print(f"Loaded {len(self.samples)} samples for {self.task} classification")
        
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        
        try:
            # Load and convert image
            image = Image.open(img_path).convert('RGB')
            
            if self.transform:
                image = self.transform(image)
                
            return image, label
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")
            # Return a default image if loading fails
            default_img = Image.new('RGB', (224, 224), color=(128, 128, 128))
            if self.transform:
                default_img = self.transform(default_img)
            return default_img, label

# Define transforms based on README specifications
train_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

print("✅ Dataset class and transforms defined successfully")

## 3. Model Loading and Configuration

Let's define our ResNet-18 based model architecture for both cattle and breed classification.

In [ ]:
def create_model(num_classes, pretrained=True):
    """
    Create ResNet-18 model with transfer learning
    
    Args:
        num_classes: Number of output classes
        pretrained: Whether to use pretrained weights
    
    Returns:
        model: PyTorch model
    """
    # Load pretrained ResNet-18
    model = models.resnet18(pretrained=pretrained)
    
    # Replace the final fully connected layer
    num_features = model.fc.in_features
    model.fc = nn.Linear(num_features, num_classes)
    
    return model

# Create models for both tasks
print("Creating models...")

# Cattle classifier (3 classes: Cow, Buffalo, None)
cattle_model = create_model(num_classes=3, pretrained=True)
cattle_model.to(device)

# Breed classifier (41 classes)
breed_model = create_model(num_classes=41, pretrained=True)
breed_model.to(device)

print("✅ Models created successfully")
print(f"Cattle model: {sum(p.numel() for p in cattle_model.parameters())} parameters")
print(f"Breed model: {sum(p.numel() for p in breed_model.parameters())} parameters")

# Check if existing models exist
cattle_model_path = models_dir / 'best_cow_buffalo_none_classifier.pth'
breed_model_path = models_dir / 'breed_classifier.pth'

if cattle_model_path.exists():
    print(f"✅ Found existing cattle model: {cattle_model_path}")
if breed_model_path.exists():
    print(f"✅ Found existing breed model: {breed_model_path}")

# Model summary function
def print_model_summary(model, model_name):
    """Print model architecture summary"""
    print(f"\n{model_name} Architecture:")
    print("=" * 50)
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Total parameters: {total_params:,}")
    print(f"Trainable parameters: {trainable_params:,}")
    print(f"Model size: {total_params * 4 / 1024 / 1024:.2f} MB")

print_model_summary(cattle_model, "Cattle Classifier")
print_model_summary(breed_model, "Breed Classifier")

## 4. Training Configuration and Hyperparameters

Set up training parameters, learning rates, and other hyperparameters as specified in the README.

In [ ]:
# Training Configuration
TRAINING_CONFIG = {
    'batch_size': 32,
    'learning_rate': 0.001,
    'num_epochs': 50,
    'patience': 10,  # Early stopping patience
    'min_delta': 0.001,  # Minimum change to qualify as improvement
    'train_split': 0.8,  # 80% for training, 20% for validation
    'weight_decay': 1e-4,
    'step_size': 10,  # Learning rate scheduler step size
    'gamma': 0.1,  # Learning rate decay factor
}

print("Training Configuration:")
print("=" * 30)
for key, value in TRAINING_CONFIG.items():
    print(f"{key}: {value}")

# Create training helper functions
def create_optimizer_and_scheduler(model, config):
    """Create optimizer and learning rate scheduler"""
    optimizer = optim.Adam(
        model.parameters(), 
        lr=config['learning_rate'],
        weight_decay=config['weight_decay']
    )
    
    scheduler = optim.lr_scheduler.StepLR(
        optimizer,
        step_size=config['step_size'],
        gamma=config['gamma']
    )
    
    return optimizer, scheduler

def create_criterion():
    """Create loss function"""
    return nn.CrossEntropyLoss()

# Create data loaders function
def create_data_loaders(dataset, config):
    """Create training and validation data loaders"""
    # Split dataset
    train_size = int(config['train_split'] * len(dataset))
    val_size = len(dataset) - train_size
    
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
    
    # Apply different transforms
    train_dataset.dataset.transform = train_transforms
    val_dataset.dataset.transform = val_transforms
    
    # Create data loaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=config['batch_size'],
        shuffle=True,
        num_workers=0,  # Set to 0 for Windows compatibility
        pin_memory=True
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=config['batch_size'],
        shuffle=False,
        num_workers=0,
        pin_memory=True
    )
    
    print(f"Training samples: {len(train_dataset)}")
    print(f"Validation samples: {len(val_dataset)}")
    
    return train_loader, val_loader

print("✅ Training configuration and helper functions defined")

## 5. Model Training Loop

Implement the comprehensive training loop with proper loss calculation, backpropagation, and optimization steps.

In [ ]:
def train_model(model, train_loader, val_loader, model_name, config):
    """
    Train model with early stopping and best model saving
    
    Args:
        model: PyTorch model to train
        train_loader: Training data loader
        val_loader: Validation data loader
        model_name: Name for saving the model
        config: Training configuration dictionary
    
    Returns:
        model: Trained model
        history: Training history
    """
    
    # Initialize training components
    criterion = create_criterion()
    optimizer, scheduler = create_optimizer_and_scheduler(model, config)
    
    # Training history
    history = {
        'train_loss': [],
        'train_acc': [],
        'val_loss': [],
        'val_acc': []
    }
    
    # Early stopping variables
    best_val_acc = 0.0
    patience_counter = 0
    best_model_state = None
    
    print(f"Starting training for {model_name}")
    print("=" * 50)
    
    for epoch in range(config['num_epochs']):
        print(f"Epoch {epoch+1}/{config['num_epochs']}")
        print("-" * 30)
        
        # Training phase
        model.train()
        running_train_loss = 0.0
        train_correct = 0
        train_total = 0
        
        train_pbar = tqdm(train_loader, desc="Training", leave=False)
        for batch_idx, (images, labels) in enumerate(train_pbar):
            images, labels = images.to(device), labels.to(device)
            
            # Zero gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            # Backward pass
            loss.backward()
            optimizer.step()
            
            # Statistics
            running_train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()
            
            # Update progress bar
            train_pbar.set_postfix({
                'Loss': f'{loss.item():.4f}',
                'Acc': f'{100 * train_correct / train_total:.2f}%'
            })
        
        # Calculate training metrics
        train_loss = running_train_loss / len(train_loader)
        train_acc = 100 * train_correct / train_total
        
        # Validation phase
        model.eval()
        running_val_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            val_pbar = tqdm(val_loader, desc="Validation", leave=False)
            for images, labels in val_pbar:
                images, labels = images.to(device), labels.to(device)
                
                outputs = model(images)
                loss = criterion(outputs, labels)
                
                running_val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
                
                val_pbar.set_postfix({
                    'Loss': f'{loss.item():.4f}',
                    'Acc': f'{100 * val_correct / val_total:.2f}%'
                })
        
        # Calculate validation metrics
        val_loss = running_val_loss / len(val_loader)
        val_acc = 100 * val_correct / val_total
        
        # Update learning rate
        scheduler.step()
        
        # Save metrics
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        
        # Print epoch results
        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
        print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")
        print(f"Learning Rate: {optimizer.param_groups[0]['lr']:.6f}")
        
        # Early stopping check
        if val_acc > best_val_acc + config['min_delta']:
            best_val_acc = val_acc
            patience_counter = 0
            best_model_state = model.state_dict().copy()
            print(f"💾 New best model saved! Validation Accuracy: {best_val_acc:.2f}%")
        else:
            patience_counter += 1
            print(f"⏳ Patience: {patience_counter}/{config['patience']}")
            
        if patience_counter >= config['patience']:
            print(f"🛑 Early stopping triggered after {epoch + 1} epochs")
            break
            
        print()
    
    # Load best model
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        print(f"✅ Best model restored with validation accuracy: {best_val_acc:.2f}%")
    
    return model, history

print("✅ Training function defined successfully")

In [ ]:
# Training execution example (uncomment to run actual training)
# Note: This cell demonstrates how to run training for both models

def run_training_pipeline():
    """Complete training pipeline for both cattle and breed classification"""
    
    print("🚀 Starting Complete Training Pipeline")
    print("=" * 60)
    
    # Check if dataset exists
    if not dataset_dir.exists():
        print("❌ Dataset not found. Please download the dataset first.")
        print("Instructions:")
        print("1. Set up Kaggle API credentials")
        print("2. Run: !kaggle datasets download -d lukex9442/indian-bovine-breeds")
        print("3. Run: !unzip -q indian-bovine-breeds.zip -d ./data/")
        return
    
    # Step 1: Train Cattle Classifier (Cow vs Buffalo vs None)
    print("\n🐄 Step 1: Training Cattle Classifier")
    print("-" * 40)
    
    try:
        # Create cattle dataset
        cattle_dataset = CattleBreedDataset(dataset_dir, task='cattle')
        cattle_train_loader, cattle_val_loader = create_data_loaders(cattle_dataset, TRAINING_CONFIG)
        
        # Train cattle model
        trained_cattle_model, cattle_history = train_model(
            cattle_model, 
            cattle_train_loader, 
            cattle_val_loader, 
            "Cattle Classifier",
            TRAINING_CONFIG
        )
        
        # Save cattle model
        cattle_save_path = models_dir / 'best_cow_buffalo_none_classifier.pth'
        torch.save(trained_cattle_model.state_dict(), cattle_save_path)
        print(f"✅ Cattle model saved to {cattle_save_path}")
        
    except Exception as e:
        print(f"❌ Error training cattle classifier: {e}")
    
    # Step 2: Train Breed Classifier
    print("\n🐂 Step 2: Training Breed Classifier")
    print("-" * 40)
    
    try:
        # Create breed dataset
        breed_dataset = CattleBreedDataset(dataset_dir, task='breed')
        breed_train_loader, breed_val_loader = create_data_loaders(breed_dataset, TRAINING_CONFIG)
        
        # Train breed model
        trained_breed_model, breed_history = train_model(
            breed_model,
            breed_train_loader,
            breed_val_loader,
            "Breed Classifier",
            TRAINING_CONFIG
        )
        
        # Save breed model
        breed_save_path = models_dir / 'breed_classifier.pth'
        torch.save(trained_breed_model.state_dict(), breed_save_path)
        print(f"✅ Breed model saved to {breed_save_path}")
        
    except Exception as e:
        print(f"❌ Error training breed classifier: {e}")
    
    print("\n🎉 Training Pipeline Complete!")
    return cattle_history, breed_history

# Uncomment the line below to run the complete training pipeline
# cattle_history, breed_history = run_training_pipeline()

print("✅ Training pipeline function defined")
print("💡 Uncomment the last line to run actual training")

## 6. Model Evaluation and Metrics

Evaluate model performance using appropriate metrics and validation techniques.

In [ ]:
def evaluate_model(model, data_loader, class_names, model_name):
    """
    Comprehensive model evaluation with metrics and visualizations
    
    Args:
        model: Trained PyTorch model
        data_loader: Data loader for evaluation
        class_names: List of class names
        model_name: Name of the model for display
    
    Returns:
        metrics: Dictionary containing evaluation metrics
    """
    model.eval()
    
    all_predictions = []
    all_labels = []
    all_probabilities = []
    
    print(f"🔍 Evaluating {model_name}")
    print("=" * 50)
    
    with torch.no_grad():
        for images, labels in tqdm(data_loader, desc="Evaluating"):
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            probabilities = torch.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs, 1)
            
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probabilities.extend(probabilities.cpu().numpy())
    
    # Convert to numpy arrays
    all_predictions = np.array(all_predictions)
    all_labels = np.array(all_labels)
    all_probabilities = np.array(all_probabilities)
    
    # Calculate metrics
    accuracy = accuracy_score(all_labels, all_predictions)
    
    # Top-3 accuracy (for breed classification)
    if len(class_names) > 3:
        top3_correct = 0
        for i in range(len(all_labels)):
            true_label = all_labels[i]
            top3_pred = np.argsort(all_probabilities[i])[-3:]
            if true_label in top3_pred:
                top3_correct += 1
        top3_accuracy = top3_correct / len(all_labels)
    else:
        top3_accuracy = accuracy
    
    print(f"📊 {model_name} Results:")
    print(f"Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
    print(f"Top-3 Accuracy: {top3_accuracy:.4f} ({top3_accuracy*100:.2f}%)")
    
    # Classification report
    print(f"\n📋 Classification Report:")
    report = classification_report(all_labels, all_predictions, 
                                  target_names=class_names, 
                                  zero_division=0)
    print(report)
    
    # Confusion Matrix
    cm = confusion_matrix(all_labels, all_predictions)
    
    # Plot confusion matrix
    plt.figure(figsize=(12, 10))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.title(f'{model_name} - Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()
    
    # Return metrics
    metrics = {
        'accuracy': accuracy,
        'top3_accuracy': top3_accuracy,
        'predictions': all_predictions,
        'labels': all_labels,
        'probabilities': all_probabilities,
        'confusion_matrix': cm,
        'classification_report': report
    }
    
    return metrics

def plot_training_history(history, model_name):
    """Plot training history"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Plot loss
    ax1.plot(history['train_loss'], label='Training Loss', marker='o')
    ax1.plot(history['val_loss'], label='Validation Loss', marker='s')
    ax1.set_title(f'{model_name} - Training Loss')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Plot accuracy
    ax2.plot(history['train_acc'], label='Training Accuracy', marker='o')
    ax2.plot(history['val_acc'], label='Validation Accuracy', marker='s')
    ax2.set_title(f'{model_name} - Training Accuracy')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy (%)')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

def load_and_evaluate_existing_models():
    """Load and evaluate existing trained models"""
    
    print("🔍 Loading and Evaluating Existing Models")
    print("=" * 60)
    
    # Load Cattle Classifier
    if cattle_model_path.exists():
        try:
            cattle_model.load_state_dict(torch.load(cattle_model_path, map_location=device))
            print("✅ Cattle classifier loaded successfully")
            
            # Create test dataset (you would need actual test data)
            # For demonstration, we'll use a subset of training data
            print("📊 Cattle Classifier Performance:")
            print("(Note: Using training subset - in practice, use separate test set)")
            
        except Exception as e:
            print(f"❌ Error loading cattle classifier: {e}")
    
    # Load Breed Classifier
    if breed_model_path.exists():
        try:
            breed_model.load_state_dict(torch.load(breed_model_path, map_location=device))
            print("✅ Breed classifier loaded successfully")
            
            print("📊 Breed Classifier Performance:")
            print("(Note: Using training subset - in practice, use separate test set)")
            
        except Exception as e:
            print(f"❌ Error loading breed classifier: {e}")
    
    return cattle_model, breed_model

# Load existing models if available
loaded_cattle_model, loaded_breed_model = load_and_evaluate_existing_models()

print("✅ Evaluation functions defined successfully")

## 7. Model Saving and Checkpoints

Save trained model weights and create checkpoints for future use in production.

In [ ]:
def save_model_checkpoint(model, optimizer, epoch, loss, accuracy, filepath):
    """
    Save comprehensive model checkpoint
    
    Args:
        model: PyTorch model
        optimizer: Optimizer state
        epoch: Current epoch
        loss: Current loss
        accuracy: Current accuracy
        filepath: Path to save checkpoint
    """
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
        'accuracy': accuracy,
        'timestamp': torch.tensor(pd.Timestamp.now().timestamp())
    }
    
    torch.save(checkpoint, filepath)
    print(f"✅ Checkpoint saved: {filepath}")

def load_model_checkpoint(model, optimizer, filepath):
    """
    Load model checkpoint
    
    Args:
        model: PyTorch model
        optimizer: Optimizer
        filepath: Path to checkpoint
    
    Returns:
        epoch: Loaded epoch
        loss: Loaded loss
        accuracy: Loaded accuracy
    """
    checkpoint = torch.load(filepath, map_location=device)
    
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    accuracy = checkpoint['accuracy']
    
    print(f"✅ Checkpoint loaded: {filepath}")
    print(f"Epoch: {epoch}, Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")
    
    return epoch, loss, accuracy

def save_production_models():
    """Save final production-ready models"""
    
    print("💾 Saving Production Models")
    print("=" * 40)
    
    # Create models directory if it doesn't exist
    models_dir.mkdir(exist_ok=True)
    
    # Save Cattle Classifier
    cattle_save_path = models_dir / 'best_cow_buffalo_none_classifier.pth'
    try:
        # Save only the state dict for production
        torch.save(cattle_model.state_dict(), cattle_save_path)
        print(f"✅ Cattle classifier saved: {cattle_save_path}")
        
        # Save model info
        cattle_info = {
            'model_type': 'ResNet18',
            'num_classes': 3,
            'classes': CATTLE_CLASSES,
            'input_size': (224, 224),
            'accuracy_target': '95%+',
            'confidence_threshold': 0.6
        }
        
        info_path = models_dir / 'cattle_model_info.json'
        with open(info_path, 'w') as f:
            import json
            json.dump(cattle_info, f, indent=2)
        
    except Exception as e:
        print(f"❌ Error saving cattle classifier: {e}")
    
    # Save Breed Classifier
    breed_save_path = models_dir / 'breed_classifier.pth'
    try:
        torch.save(breed_model.state_dict(), breed_save_path)
        print(f"✅ Breed classifier saved: {breed_save_path}")
        
        # Save model info
        breed_info = {
            'model_type': 'ResNet18',
            'num_classes': 41,
            'classes': ALL_BREEDS,
            'indian_breeds': INDIAN_BREEDS,
            'international_breeds': INTERNATIONAL_BREEDS,
            'input_size': (224, 224),
            'accuracy_target': '88%+',
            'top3_accuracy_target': '96%+'
        }
        
        info_path = models_dir / 'breed_model_info.json'
        with open(info_path, 'w') as f:
            import json
            json.dump(breed_info, f, indent=2)
            
    except Exception as e:
        print(f"❌ Error saving breed classifier: {e}")
    
    # Create model summary
    summary = {
        'project': 'Cattle & Breed Classification',
        'framework': 'PyTorch',
        'architecture': 'ResNet-18',
        'training_date': pd.Timestamp.now().isoformat(),
        'models': {
            'cattle_classifier': {
                'file': 'best_cow_buffalo_none_classifier.pth',
                'classes': 3,
                'task': 'Cattle species classification'
            },
            'breed_classifier': {
                'file': 'breed_classifier.pth', 
                'classes': 41,
                'task': 'Breed identification'
            }
        },
        'dataset_source': 'https://www.kaggle.com/datasets/lukex9442/indian-bovine-breeds',
        'deployment_ready': True
    }
    
    summary_path = models_dir / 'model_summary.json'
    with open(summary_path, 'w') as f:
        import json
        json.dump(summary, f, indent=2)
    
    print(f"✅ Model summary saved: {summary_path}")
    print("\n🎉 All production models saved successfully!")

def test_model_loading():
    """Test loading saved models to ensure they work correctly"""
    
    print("🧪 Testing Model Loading")
    print("=" * 30)
    
    # Test cattle model loading
    try:
        test_cattle_model = create_model(num_classes=3, pretrained=False)
        test_cattle_model.load_state_dict(torch.load(cattle_model_path, map_location=device))
        test_cattle_model.eval()
        print("✅ Cattle model loading test passed")
    except Exception as e:
        print(f"❌ Cattle model loading test failed: {e}")
    
    # Test breed model loading
    try:
        test_breed_model = create_model(num_classes=41, pretrained=False)
        test_breed_model.load_state_dict(torch.load(breed_model_path, map_location=device))
        test_breed_model.eval()
        print("✅ Breed model loading test passed")
    except Exception as e:
        print(f"❌ Breed model loading test failed: {e}")
    
    print("\n🎯 Models ready for production deployment!")

# Save production models (if training was completed)
save_production_models()

# Test model loading
test_model_loading()

print("✅ Model saving and checkpoint functions defined successfully")

## 🎯 Training Summary and Next Steps

### What This Notebook Provides:

1. **Complete Training Pipeline**: Ready-to-use training code for both cattle and breed classification models
2. **Data Management**: Automatic dataset downloading and preprocessing from Kaggle
3. **Model Architecture**: ResNet-18 with transfer learning optimized for cattle classification
4. **Training Features**:
   - Early stopping to prevent overfitting
   - Learning rate scheduling
   - Comprehensive data augmentation
   - Real-time progress tracking
   - Automatic best model saving

5. **Evaluation Tools**: 
   - Accuracy metrics and confusion matrices
   - Classification reports
   - Top-3 accuracy for breed classification
   - Training history visualization

6. **Production Ready**: Models saved in format compatible with the Streamlit application

### 📋 To Run Training:

1. **Setup Kaggle API**:
   ```bash
   pip install kaggle
   # Get kaggle.json from kaggle.com/account
   # Place in ~/.kaggle/ directory
   ```

2. **Download Dataset**: Uncomment and run the dataset download cells

3. **Start Training**: Uncomment the training pipeline execution line

4. **Monitor Progress**: Use the built-in progress bars and metrics

### 🎯 Expected Results:

- **Cattle Classifier**: 95%+ accuracy (Cow vs Buffalo vs None)
- **Breed Classifier**: 88%+ accuracy (41 breeds), 96%+ top-3 accuracy
- **Model Files**: Saved to `models/` directory for deployment

### 🚀 Ready for Production:

The trained models will be saved as:
- `models/best_cow_buffalo_none_classifier.pth`
- `models/breed_classifier.pth`

These files are directly compatible with the Streamlit application in `cattle_with_breed_classifier.py`!